In [62]:
import requests
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import os
import datetime
import pandas as pd


In [63]:
def get_historical_data(asteroid_id):
    url = f"https://api.nasa.gov/neo/rest/v1/neo/{asteroid_id}?api_key={os.getenv('NASA_API_KEY')}"
    response = requests.get(url)
    data = response.json()
    return data.get('close_approach_data')

In [64]:
def extract_features(close_approach_data):
    features = []
    for approach in close_approach_data:
        date = approach['close_approach_date']
        velocity = float(approach['relative_velocity']['kilometers_per_second'])
        features.append({
            'date': date,
            'velocity': velocity
        })
    return features


In [65]:
sound_data = extract_features(get_historical_data('3542519'))
sound_data

[{'date': '1900-06-01', 'velocity': 30.9354328365},
 {'date': '1900-07-07', 'velocity': 31.7784421852},
 {'date': '1900-07-27', 'velocity': 23.864674044},
 {'date': '1902-02-11', 'velocity': 28.1657402294},
 {'date': '1903-09-02', 'velocity': 20.7057263268},
 {'date': '1904-07-26', 'velocity': 15.8485675157},
 {'date': '1907-04-13', 'velocity': 30.3958592282},
 {'date': '1907-08-23', 'velocity': 30.7218238305},
 {'date': '1908-08-17', 'velocity': 14.3282384178},
 {'date': '1909-07-31', 'velocity': 26.865742776},
 {'date': '1912-09-02', 'velocity': 18.1653406437},
 {'date': '1913-07-24', 'velocity': 17.6679368771},
 {'date': '1914-01-17', 'velocity': 28.8053254859},
 {'date': '1915-09-12', 'velocity': 25.4093966639},
 {'date': '1916-08-26', 'velocity': 27.5358559955},
 {'date': '1917-05-22', 'velocity': 29.919887598},
 {'date': '1917-08-09', 'velocity': 14.3091269675},
 {'date': '1918-08-04', 'velocity': 30.2107926533},
 {'date': '1919-01-29', 'velocity': 34.3398497229},
 {'date': '1920

In [66]:
# Sonification:
# - Date: pitch
# - Velocity: volume

# 1. Normalize the velocity
# 2. Map the date to a pitch
# 3. Map the velocity to a volume
# 4. Play the sonification


In [67]:
import pandas as pd
import numpy as np
from pydub import AudioSegment
from pydub.generators import Sine, Square, Sawtooth
import random


def normalize(data):
    """Normalize data to the range [0, 1]."""
    min_val = np.min(data)
    max_val = np.max(data)
    return (data - min_val) / (max_val - min_val)


# Prepare data for sonification
def prepare_sonification_data(data):
    df = pd.DataFrame(data)
    
    # Convert date to datetime and extract year
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    
    # Convert velocity to float
    df['velocity'] = df['velocity'].astype(float)
    
    # Normalize velocity
    df['normalized_velocity'] = normalize(df['velocity'])
    
    return df

def generate_tone(frequency, duration_ms, volume_db):
    wave_forms = ['sine', 'square', 'sawtooth']
    random_int = random.randint(0,2)
    waveform = wave_forms[random_int]
    if waveform == 'sine':
        tone = Sine(frequency).to_audio_segment(duration=duration_ms)
    elif waveform == 'square':
        tone = Square(frequency).to_audio_segment(duration=duration_ms)
    elif waveform == 'sawtooth':
        tone = Sawtooth(frequency).to_audio_segment(duration=duration_ms)
    tone = Sine(frequency).to_audio_segment(duration=duration_ms)
    tone = tone + volume_db  # Adjust volume
    return tone
def create_sonification(df):
    min_year = df['year'].min()
    max_year = df['year'].max()
    min_velocity = df['normalized_velocity'].min()
    max_velocity = df['normalized_velocity'].max()
    
    # Initialize empty audio segment
    combined_audio = AudioSegment.silent(duration=0)
    
    for index, row in df.iterrows():
        year = row['year']
        velocity_normalized = row['normalized_velocity']
        
        # Map year to frequency (in Hz)
        frequency = np.interp(year, [min_year, max_year], [10, 1000])  
        
        # Map normalized velocity to volume (in dB)
        volume_db = np.interp(velocity_normalized, [min_velocity, max_velocity], [-20, 0])  
        
        # Duration in milliseconds
        duration_ms = 16  # Duration for each tone
        
        # Generate tone
        tone = generate_tone(frequency, duration_ms, volume_db)
        
        # Append tone to combined audio segment
        combined_audio += tone
    
    # Export combined audio track
    combined_audio.export('../frontend/sound_data/combined_sonification.mp3', format='mp3')

# Example data
data = [
    {'date': '2183-07-27', 'velocity': '18.6029484078'},
    {'date': '2184-01-07', 'velocity': '39.1465158456'},
    {'date': '2186-08-30', 'velocity': '26.3953248467'},
    {'date': '2187-08-09', 'velocity': '14.3703716599'},
    {'date': '2188-06-18', 'velocity': '43.0627895674'},
    {'date': '2188-08-07', 'velocity': '31.2964399593'},
    {'date': '2190-02-26', 'velocity': '34.479962683'},
    {'date': '2190-04-08', 'velocity': '24.9795023853'},
    {'date': '2191-09-02', 'velocity': '15.3625591818'},
]
# Sample usage
# df = prepare_sonification_data(sound_data)
# create_sonification(df)
